# 작업 1유형

## 데이터 로드

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')

## 마케팅 응답 고객들의 나이를 10살 단위로 변환 햿을 때, 가장 많은 인원을 가진 나이대

In [8]:
df['age_range'] = df['age'].map(lambda x : x//10 * 10)
print((df['age_range'].value_counts().index[0]))

30


## 마케팅 응답 고객들의 나이를 10살 단위로 변환 헀을 때, 가장 많은 나이대 구간의 인원

In [10]:
print((df['age_range'].value_counts().values[0]))

5056


## 나이가 25살 이상 29살 미만인 응답 고객들 중 housing 컬럼의 값이 yes인 고객의 수

In [14]:
print(len(df.loc[(df.age >= 25)&(df.age< 29)&(df.housing=='yes')]))

504


## numeric한 값을 가지지 않은 컬럼들 중 unique한 값을 가장 많이 가지는 컬럼

In [26]:
print(df.describe(include='object').loc['unique',:].sort_values(ascending=False).index[0],df.describe(include='object').loc['unique',:].sort_values(ascending=False).index[1])

job month


## balance 컬럼값들의 평균값 이상을 가지는 데이터를 ID 값을 기준으로 내림차순 정렬했을 때 상위 100개 데이터의 balance 평균

In [31]:
print(df.loc[df.balance >= df.balance.mean()].sort_values('ID',ascending=False)[:100].balance.mean())

3473.73


## 가장 많은 광고를 집행했던 날짜

In [45]:
print(df.groupby(['month','day']).size().sort_values(ascending=False).reset_index().loc[0,['month','day']])

month    may
day       15
Name: 0, dtype: object


## 데이터의 job이 unknown 상태인 고객들의 age 컬럼 값의 정규성 검정

In [49]:
from scipy import stats 
statistic, pvalue = stats.shapiro(df.loc[df.job=='unknown'].age)
print(pvalue)

0.1961223594771469


## age와 balance의 상관계수 

In [61]:
import numpy as np
print(df[['age','balance']].corr().iloc[0,1])

0.10198734763981462


## y변수와 education 변수는 독립인지 카이제곱검정을 통해 p-value 확인

In [66]:
df_matrix = pd.crosstab(df.y,df.education)
statistic,pvalue,dof,expected = stats.chi2_contingency(df_matrix)
print(pvalue)

7.901201277473551e-29


## 각 job에 따라 divorced/married 인원의 비율을 확인했을 때 그 값이 가장 높은 값

In [82]:
df.marital.unique()
result = df.groupby(['job','marital']).size().unstack()
result['ratio'] = result.divorced/result.married
print(result.sort_values('ratio',ascending=False).ratio.values[0])

0.2831050228310502


# 작업 2유형

In [112]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score 
from sklearn.ensemble import RandomForestClassifier 
train= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/train.csv')
test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/test.csv')
submission= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bank/submission.csv')

for i in train.describe(include='object').columns:
    train[i] = LabelEncoder().fit_transform(train[i])
for i in test.describe(include='object').columns:
    test[i] = LabelEncoder().fit_transform(test[i])
test.drop(['ID'],axis=1,inplace=True)
feature = train.drop(['ID','y'],axis=1)
target = train['y']

print(feature.describe(include='all'))
print('')
print(target.describe(include='all'))

ftr,fte,ttr,tte = train_test_split(feature,target, test_size= 0.33, random_state = 23)

RFC = RandomForestClassifier(random_state = 23)
RFC.fit(ftr,ttr)
pred = RFC.predict(fte)
pred_proba = RFC.predict_proba(fte)[:,1]

acc = accuracy_score(tte,pred)
f1 = f1_score(tte,pred)
precision = precision_score(tte,pred)
recall = recall_score(tte,pred)
auc = roc_auc_score(tte,pred_proba)
print(acc)
print(f1)
print(precision)
print(recall)
print(auc)

pred = RFC.predict(test)
pred_proba = RFC.predict_proba(test)
submission['predict'] = pred_proba[:,1]
print(test)
print(submission)

                age           job       marital     education       default  \
count  12870.000000  12870.000000  12870.000000  12870.000000  12870.000000   
mean      41.091142      4.413287      1.188811      1.255556      0.016162   
std       11.305560      3.254204      0.618390      0.753291      0.126102   
min       18.000000      0.000000      0.000000      0.000000      0.000000   
25%       32.000000      1.000000      1.000000      1.000000      0.000000   
50%       39.000000      4.000000      1.000000      1.000000      0.000000   
75%       49.000000      7.000000      2.000000      2.000000      0.000000   
max       95.000000     11.000000      2.000000      3.000000      1.000000   

             balance       housing          loan       contact           day  \
count   12870.000000  12870.000000  12870.000000  12870.000000  12870.000000   
mean     1483.774437      0.513209      0.140637      0.571329     15.641103   
std      3311.055181      0.499845      0.347660